In [1]:
 from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re  # Importing the regular expression module
from datetime import datetime  # Importing datetime for timestamp
from selenium.webdriver.chrome.service import Service

# Initialize Chrome driver with Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())


url = "https://www.ah.nl/producten/snoep-chocolade-koek?merk=Delicata&soort=8729&page=1"
driver.get(url)
time.sleep(5)

# Borrowed the 'very cool :)' accepteer cookies button from TOTO scraper(group assignment)
accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "decline-cookies")))
accept_button.click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

time.sleep(5)

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('article', class_='product-card-portrait_root__ZiRpZ'):
    # Extract the price from the aria-label of the sr-only span
    price_span = article.find('span', class_='sr-only')
    if price_span:
        # Use regular expression to extract the numeric price (e.g., 1.99)
        match = re.search(r'[\d]+[.,][\d]+', price_span.get('aria-label'))
        price = match.group() if match else 'Price not found'
    else:
        price = 'Price not found'
        
    # Extract the promo price (if available) from the correct div
    promo_price_span = article.find('div', class_='price-amount_highlight__ekL92')
    if promo_price_span:
        # Use a nested find to get the sr-only span within the promo price div
        promo_price_span_inner = promo_price_span.find('span', class_='sr-only')
        if promo_price_span_inner:
            # Use regular expression to extract the numeric promo price (e.g., 6.53)
            match_promo_price = re.search(r'[\d]+[.,][\d]+', promo_price_span_inner.get('aria-label'))
            promo_price = match_promo_price.group() if match_promo_price else 'Promo price not found'
        else:
            promo_price = 'Promo price not found'
    else:
        promo_price = 'Promo price not found'

    # Extract the product title from the title attribute of the anchor tag
    title_tag = article.find('a', class_='link_root__EqRHd')
    title = title_tag.get('title') if title_tag else 'Title not found'
    
    # Extract the weight from the product-unit-size span
    weight_span = article.find('span', class_='price_unitSize__Hk6E4')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'

    # Store the extracted information as a tuple, including promo price
    products.append((title, price, promo_price, weight))
    
# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD HH:MM:SS

# Write the data to a CSV file
with open('repen.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Product Title', 'Price', 'promo_price', 'weight', 'Timestamp'])  # Write header row
    for product in products:
        writer.writerow((*product, timestamp))  # Write product data with timestamp

print(f"Data has been successfully saved to products.csv")
driver.quit()

Data has been successfully saved to products.csv


In [2]:


# Initialize Chrome driver with Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())

url = "https://www.ah.nl/producten/snoep-chocolade-koek?merk=AH&soort=8729"
driver.get(url)
time.sleep(5)

# Borrowed the 'very cool :)' accepteer cookies button from TOTO scraper(group assignment)
accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "decline-cookies")))
accept_button.click()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

time.sleep(5)

# List to store the extracted product information
products = []

# Loop through all product articles
for article in soup.find_all('article', class_='product-card-portrait_root__ZiRpZ'):
    # Extract the price from the aria-label of the sr-only span
    price_span = article.find('span', class_='sr-only')
    if price_span:
        # Use regular expression to extract the numeric price (e.g., 1.99)
        match = re.search(r'[\d]+[.,][\d]+', price_span.get('aria-label'))
        price = match.group() if match else 'Price not found'
    else:
        price = 'Price not found'
        
    promo_price_span = article.find('div', class_='price-amount_highlight__ekL92').find('span', class_='sr-only')
    if promo_price_span:
        # Use regular expression to extract the numeric current price (e.g., 6.53)
        match_promo_price = re.search(r'[\d]+[.,][\d]+', promo_price_span.get('aria-label'))
        promo_price = match_promo_price.group() if match_promo_price else 'Current price not found'
    else:
        promo_price = 'Promo price not found'

    # Extract the product title from the title attribute of the anchor tag
    title_tag = article.find('a', class_='link_root__EqRHd')
    title = title_tag.get('title') if title_tag else 'Title not found'
    
    # Extract the weight from the product-unit-size span
    weight_span = article.find('span', class_='price_unitSize__Hk6E4')
    weight = weight_span.get_text(strip=True) if weight_span else 'Weight not found'


    # Store the extracted information as a tuple
    products.append((title, price, promo_price, weight))

# Get current timestamp for the data
timestamp = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD HH:MM:SS

# Write the data to a CSV file
with open('repen.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for product in products:
        writer.writerow((*product, timestamp)) 

print(f"Data has been successfully saved to products.csv")
driver.quit()

Data has been successfully saved to products.csv
